# Notebook 02: Processamento de Dados Médicos

Este notebook processa os dados médicos carregados no notebook anterior. Aqui vamos:

1. **Anonimizar** dados sensíveis (conformidade LGPD/HIPAA)
2. **Processar** cada entrada do dataset
3. **Limpar** e normalizar textos
4. **Dividir** textos em chunks otimizados
5. **Validar** qualidade dos dados processados

## 📋 Pré-requisitos

- Notebook 01 executado com sucesso (dataset carregado)
- Variáveis de ambiente configuradas
- Dependências instaladas

## 🔄 Ordem de Execução

Execute as células **sequencialmente** (de cima para baixo).


In [6]:
# ============================================================================
# ETAPA 0: IMPORTAÇÃO DE BIBLIOTECAS E CONFIGURAÇÕES
# ============================================================================
# Esta célula importa todas as bibliotecas necessárias e configura o ambiente
# Execute esta célula PRIMEIRO antes de qualquer outra operação

import sys
import importlib
from pathlib import Path

# Adiciona o diretório raiz ao path para importar módulos
# Quando executado do diretório notebooks/, o parent é rag_medical/
root_dir = Path.cwd().parent
if str(root_dir) not in sys.path:
    sys.path.insert(0, str(root_dir))

# Recarrega módulos se já foram importados (útil durante desenvolvimento)
# IMPORTANTE: Recarrega na ordem correta (dependências primeiro)
if 'utils.anonymizer' in sys.modules:
    importlib.reload(sys.modules['utils.anonymizer'])
if 'utils' in sys.modules:
    importlib.reload(sys.modules['utils'])
if 'scripts.data_processor' in sys.modules:
    importlib.reload(sys.modules['scripts.data_processor'])
if 'scripts.data_loader' in sys.modules:
    importlib.reload(sys.modules['scripts.data_loader'])
if 'config.settings' in sys.modules:
    importlib.reload(sys.modules['config.settings'])
if 'config' in sys.modules:
    importlib.reload(sys.modules['config'])

# Importa módulos do pipeline RAG
from scripts.data_loader import load_medical_dataset
from scripts.data_processor import (
    process_medical_entry,
    process_batch,
    filter_valid_entries
)
from config.settings import get_settings

# Tenta importar text_splitter (pode falhar se dependências não estiverem instaladas)
try:
    from scripts.text_splitter import MedicalTextSplitter, create_text_splitter
except ImportError as e:
    print(f"⚠️  Aviso: text_splitter não disponível: {e}")
    print("   Você pode continuar sem ele, mas não poderá dividir textos em chunks")
    MedicalTextSplitter = None
    create_text_splitter = None

# Carrega configurações
settings = get_settings()

# Valida configurações básicas (strict=False permite exploração sem Pinecone/Embeddings)
is_valid, errors = settings.validate(strict=False)
if not is_valid:
    print("❌ ERROS DE CONFIGURAÇÃO:")
    for error in errors:
        print(f"   - {error}")
    print("\n💡 Configure as variáveis de ambiente no arquivo .env")
    raise ValueError("Configurações inválidas")
else:
    print("✅ Configurações validadas com sucesso!")
    print(f"📁 Arquivo de dados: {settings.MEDICAL_DATA_PATH}")

print("\n✅ Bibliotecas importadas com sucesso!")


✅ Configurações validadas com sucesso!
📁 Arquivo de dados: /Users/vitorteixeira/Developer/projects/tech_challenge_fase_3_v2/rag_medical/ori_pqal.json

✅ Bibliotecas importadas com sucesso!


In [7]:
# ============================================================================
# ETAPA 1: CARREGAMENTO DO DATASET
# ============================================================================
# Carrega o dataset médico. Se você já executou o notebook 01 e tem
# raw_data em memória, esta célula detectará e reutilizará.
# Caso contrário, carregará do arquivo automaticamente.

# Verifica se raw_data já está carregado (do notebook anterior)
if 'raw_data' not in globals():
    print("📂 Carregando dataset do arquivo...")
    data_path = settings.MEDICAL_DATA_PATH
    raw_data = load_medical_dataset(data_path)
    print(f"✅ Dataset carregado: {len(raw_data)} entradas")
else:
    print(f"✅ Dataset já carregado (do notebook anterior): {len(raw_data)} entradas")

print(f"📁 Arquivo: {settings.MEDICAL_DATA_PATH}")
print("=" * 80)


📂 Carregando dataset do arquivo...
✅ Dataset carregado: 1000 entradas
📁 Arquivo: /Users/vitorteixeira/Developer/projects/tech_challenge_fase_3_v2/rag_medical/ori_pqal.json


In [8]:
# ============================================================================
# ETAPA 2: PROCESSAMENTO DE DADOS MÉDICOS
# ============================================================================
# Esta etapa processa cada entrada do dataset:
# - Combina múltiplos contextos em texto único
# - Aplica anonimização de dados sensíveis
# - Formata metadados estruturados
# - Prepara dados para embedding

print("=" * 80)
print("🔄 PROCESSANDO DADOS MÉDICOS")
print("=" * 80)
print(f"Total de entradas a processar: {len(raw_data)}")
print(f"Anonimização: Habilitada (conformidade LGPD/HIPAA)")
print("-" * 80)

# Processa todas as entradas em lote
processed_entries = process_batch(
    raw_data,
    anonymize=True,  # Habilita anonimização
    show_progress=True
)

print(f"\n✅ Processamento concluído!")
print(f"   Entradas processadas: {len(processed_entries)}")
print("=" * 80)


🔄 PROCESSANDO DADOS MÉDICOS
Total de entradas a processar: 1000
Anonimização: Habilitada (conformidade LGPD/HIPAA)
--------------------------------------------------------------------------------


Processando dados: 100%|██████████| 1000/1000 [00:00<00:00, 3774.31it/s]


✅ Processamento concluído!
   Entradas processadas: 1000


In [9]:
# ============================================================================
# ETAPA 3: FILTRAGEM DE ENTRADAS VÁLIDAS
# ============================================================================
# Remove entradas muito curtas ou inválidas que não são adequadas
# para embedding e busca

print("=" * 80)
print("🔍 FILTRANDO ENTRADAS VÁLIDAS")
print("=" * 80)

total_before = len(processed_entries)
min_text_length = 50  # Tamanho mínimo em caracteres

valid_entries = filter_valid_entries(
    processed_entries,
    min_text_length=min_text_length
)

total_after = len(valid_entries)
removed = total_before - total_after

print(f"Entradas antes da filtragem: {total_before}")
print(f"Entradas após filtragem: {total_after}")
print(f"Entradas removidas: {removed} (muito curtas ou inválidas)")
print(f"Taxa de retenção: {total_after/total_before*100:.1f}%")
print("=" * 80)


🔍 FILTRANDO ENTRADAS VÁLIDAS
Entradas antes da filtragem: 1000
Entradas após filtragem: 1000
Entradas removidas: 0 (muito curtas ou inválidas)
Taxa de retenção: 100.0%


In [10]:
# ============================================================================
# ETAPA 4: DIVISÃO EM CHUNKS
# ============================================================================
# Divide textos longos em chunks menores para otimizar:
# - Embedding (chunks menores são mais eficientes)
# - Busca (recuperação mais precisa)
# - Armazenamento (melhor uso do Pinecone)

print("=" * 80)
print("✂️  DIVIDINDO TEXTOS EM CHUNKS")
print("=" * 80)

# ============================================================================
# VALIDAÇÃO: Verifica se valid_entries existe (deve ter sido criado na célula 4)
# ============================================================================
# Esta verificação DEVE ser executada ANTES de qualquer uso de valid_entries
try:
    entries_available = len(valid_entries) > 0
    if not entries_available:
        raise ValueError("valid_entries está vazio. Execute a célula 4 primeiro.")
except NameError:
    print("\n" + "=" * 80)
    print("❌ ERRO: Variável 'valid_entries' não encontrada!")
    print("=" * 80)
    print("A variável 'valid_entries' não está definida.")
    print("\n💡 SOLUÇÃO:")
    print("Execute as células anteriores na ordem:")
    print("   1. Célula 1: Importação de bibliotecas (ETAPA 0)")
    print("   2. Célula 2: Carregamento do dataset (ETAPA 1)")
    print("   3. Célula 3: Processamento de dados (ETAPA 2)")
    print("   4. Célula 4: Filtragem de entradas (ETAPA 3) ← Esta cria 'valid_entries'")
    print("   5. Célula 5: Divisão em chunks (ETAPA 4) ← Esta célula")
    print("=" * 80)
    raise NameError(
        "Variável 'valid_entries' não encontrada. "
        "Execute a célula 4 (ETAPA 3: FILTRAGEM DE ENTRADAS VÁLIDAS) primeiro."
    )

# Verifica se text_splitter está disponível
# Usa verificação segura para evitar NameError se a variável não foi definida
try:
    text_splitter_available = create_text_splitter is not None
except NameError:
    text_splitter_available = False
    print("⚠️  ATENÇÃO: Variável 'create_text_splitter' não encontrada!")
    print("   Execute a célula 1 (ETAPA 0) primeiro para importar as bibliotecas.")
    print("   Continuando sem text_splitter...")

if not text_splitter_available:
    print("⚠️  text_splitter não disponível (dependências não instaladas ou não importado)")
    print("   Pulando divisão em chunks...")
    print("   Usando entradas completas como chunks únicos")
    # Cria chunks simples (uma entrada = um chunk)
    # Nota: valid_entries já foi verificado acima, então está disponível aqui
    all_chunks = []
    for entry in valid_entries:
        chunk = {
            'text': entry['text'],
            'article_id': entry['article_id'],
            'chunk_index': 0,
            'metadata': entry['metadata']
        }
        all_chunks.append(chunk)
    print(f"\n✅ Usando entradas completas como chunks")
    print(f"   Total de chunks: {len(all_chunks)}")
else:
    print(f"Chunk size: {settings.CHUNK_SIZE} caracteres")
    print(f"Chunk overlap: {settings.CHUNK_OVERLAP} caracteres")
    print("-" * 80)
    
    # Cria divisor de texto
    text_splitter = create_text_splitter(
        chunk_size=settings.CHUNK_SIZE,
        chunk_overlap=settings.CHUNK_OVERLAP
    )
    
    # Divide todas as entradas em chunks
    all_chunks = text_splitter.split_batch(
        valid_entries,
        preserve_metadata=True,
        show_progress=True
    )
    
    print(f"\n✅ Divisão em chunks concluída!")
    print(f"   Entradas originais: {len(valid_entries)}")
    print(f"   Total de chunks gerados: {len(all_chunks)}")
    if len(valid_entries) > 0:
        print(f"   Média de chunks por entrada: {len(all_chunks)/len(valid_entries):.2f}")

print("=" * 80)


✂️  DIVIDINDO TEXTOS EM CHUNKS
Chunk size: 512 caracteres
Chunk overlap: 50 caracteres
--------------------------------------------------------------------------------


Dividindo em chunks: 100%|██████████| 1000/1000 [00:00<00:00, 20590.09it/s]


✅ Divisão em chunks concluída!
   Entradas originais: 1000
   Total de chunks gerados: 5320
   Média de chunks por entrada: 5.32


In [ ]:
# ============================================================================
# ETAPA 6: SALVAMENTO DOS CHUNKS (OPCIONAL)
# ============================================================================
# Salva os chunks processados em um arquivo JSON para uso posterior
# Isso permite carregar os chunks no notebook 03 mesmo após reiniciar o kernel

import json
from pathlib import Path

# Verifica se all_chunks existe
try:
    chunks_count = len(all_chunks)
    
    # Define o caminho do arquivo (no diretório raiz do projeto)
    chunks_file = root_dir / 'processed_chunks.json'
    
    print("=" * 80)
    print("💾 SALVANDO CHUNKS PROCESSADOS")
    print("=" * 80)
    print(f"Total de chunks a salvar: {chunks_count:,}")
    print(f"Arquivo de destino: {chunks_file}")
    print("-" * 80)
    
    # Salva os chunks em arquivo JSON
    with open(chunks_file, 'w', encoding='utf-8') as f:
        json.dump(all_chunks, f, ensure_ascii=False, indent=2)
    
    # Verifica o tamanho do arquivo
    file_size = chunks_file.stat().st_size
    file_size_mb = file_size / (1024 * 1024)
    
    print(f"✅ Chunks salvos com sucesso!")
    print(f"   Arquivo: {chunks_file}")
    print(f"   Tamanho: {file_size_mb:.2f} MB")
    print(f"   Total de chunks: {chunks_count:,}")
    print("\n💡 Agora você pode:")
    print("   - Carregar este arquivo no notebook 03")
    print("   - Compartilhar os chunks processados")
    print("   - Reutilizar sem reprocessar os dados")
    print("=" * 80)
    
except NameError:
    print("=" * 80)
    print("⚠️  ATENÇÃO: Variável 'all_chunks' não encontrada!")
    print("=" * 80)
    print("Execute a célula 5 (ETAPA 4: DIVISÃO EM CHUNKS) primeiro.")
    print("=" * 80)
except Exception as e:
    print("=" * 80)
    print(f"❌ ERRO ao salvar chunks: {e}")
    print("=" * 80)
    raise


In [11]:
# ============================================================================
# ETAPA 5: VISUALIZAÇÃO DE EXEMPLOS PROCESSADOS
# ============================================================================
# Exibe exemplos de chunks processados para verificação visual

print("=" * 80)
print("📄 EXEMPLOS DE CHUNKS PROCESSADOS")
print("=" * 80)

# Mostra alguns exemplos
for i, chunk in enumerate(all_chunks[:3], 1):
    print(f"\n{'='*80}")
    print(f"CHUNK {i}")
    print(f"{'='*80}")
    print(f"Article ID: {chunk['article_id']}")
    print(f"Chunk Index: {chunk['chunk_index']}")
    print(f"Tamanho: {len(chunk['text'])} caracteres")
    print(f"\nTexto (primeiros 300 caracteres):")
    print(f"{chunk['text'][:300]}...")
    print(f"\nMetadados:")
    for key, value in chunk['metadata'].items():
        if isinstance(value, str) and len(value) > 100:
            print(f"  {key}: {value[:100]}...")
        else:
            print(f"  {key}: {value}")

print(f"\n{'='*80}")
print("✅ Visualização concluída!")
print("=" * 80)


📄 EXEMPLOS DE CHUNKS PROCESSADOS

CHUNK 1
Article ID: 21645374
Chunk Index: 0
Tamanho: 423 caracteres

Texto (primeiros 300 caracteres):
Context: Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs i...

Metadados:
  article_id: 21645374
  question: Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?
  source: pubmedqa
  type: medical_qa
  year: 2011
  meshes: Alismataceae, Apoptosis, Cell Differentiation, Mitochondria, Plant Leaves
  labels: BACKGROUND, RESULTS
  final_decision: yes
  reasoning_required: yes
  chunk_index: 0

CHUNK 2
Article ID: 21645374
Chunk Index: 1
Tamanho: 305 caracteres

Texto (primeiros 300 caracteres):
approximately five cells from the vasculature. The role of mitochondria during PCD has 

In [12]:
# ============================================================================
# ETAPA 6: SALVAMENTO DOS CHUNKS (OPCIONAL)
# ============================================================================
# Salva os chunks processados em um arquivo JSON para uso posterior
# Isso permite carregar os chunks no notebook 03 mesmo após reiniciar o kernel

import json
from pathlib import Path

# Verifica se all_chunks existe
try:
    chunks_count = len(all_chunks)
    
    # Define o caminho do arquivo (no diretório raiz do projeto)
    chunks_file = root_dir / 'processed_chunks.json'
    
    print("=" * 80)
    print("💾 SALVANDO CHUNKS PROCESSADOS")
    print("=" * 80)
    print(f"Total de chunks a salvar: {chunks_count:,}")
    print(f"Arquivo de destino: {chunks_file}")
    print("-" * 80)
    
    # Salva os chunks em arquivo JSON
    with open(chunks_file, 'w', encoding='utf-8') as f:
        json.dump(all_chunks, f, ensure_ascii=False, indent=2)
    
    # Verifica o tamanho do arquivo
    file_size = chunks_file.stat().st_size
    file_size_mb = file_size / (1024 * 1024)
    
    print(f"✅ Chunks salvos com sucesso!")
    print(f"   Arquivo: {chunks_file}")
    print(f"   Tamanho: {file_size_mb:.2f} MB")
    print(f"   Total de chunks: {chunks_count:,}")
    print("\n💡 Agora você pode:")
    print("   - Carregar este arquivo no notebook 03")
    print("   - Compartilhar os chunks processados")
    print("   - Reutilizar sem reprocessar os dados")
    print("=" * 80)
    
except NameError:
    print("=" * 80)
    print("⚠️  ATENÇÃO: Variável 'all_chunks' não encontrada!")
    print("=" * 80)
    print("Execute a célula 5 (ETAPA 4: DIVISÃO EM CHUNKS) primeiro.")
    print("=" * 80)
except Exception as e:
    print("=" * 80)
    print(f"❌ ERRO ao salvar chunks: {e}")
    print("=" * 80)
    raise

💾 SALVANDO CHUNKS PROCESSADOS
Total de chunks a salvar: 5,320
Arquivo de destino: /Users/vitorteixeira/Developer/projects/tech_challenge_fase_3_v2/rag_medical/processed_chunks.json
--------------------------------------------------------------------------------
✅ Chunks salvos com sucesso!
   Arquivo: /Users/vitorteixeira/Developer/projects/tech_challenge_fase_3_v2/rag_medical/processed_chunks.json
   Tamanho: 5.76 MB
   Total de chunks: 5,320

💡 Agora você pode:
   - Carregar este arquivo no notebook 03
   - Compartilhar os chunks processados
   - Reutilizar sem reprocessar os dados


## ✅ Conclusão da Etapa 2

Neste notebook você:
- ✅ Processou todas as entradas do dataset
- ✅ Aplicou anonimização de dados sensíveis
- ✅ Filtrou entradas inválidas
- ✅ Dividiu textos em chunks otimizados
- ✅ Validou a qualidade dos dados processados

## 📌 Próximos Passos

Agora você está pronto para o próximo notebook:
- **Notebook 03**: Geração de embeddings e ingestão no Pinecone
- Conexão com Pinecone
- Geração de embeddings
- Ingestão em lotes
